In [65]:
import huggingface
import torch
from transformers import AutoTokenizer, BartForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import time
import datasets
from datasets import load_dataset
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import pickle


import random
import pandas as pd
from IPython.display import display, HTML


#### Load Dataset

In [4]:
dataset = load_dataset("abisee/cnn_dailymail", "1.0.0")
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

#### Check Dataset

In [5]:
def show_random_elements(dataset, num_examples=2):
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


show_random_elements(dataset["train"])

These examples show that the news articles get summurised by in a fairly succinct fashion to give just the main highlights for each. 

#### Pre-process Dataset

In [6]:
# Load model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

C:\Users\tom_r\anaconda3\envs\Generative_AI\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


In [8]:

def preprocess_function(examples):
    #inputs = ['summarize' + doc for doc in examples["article"]]
    inputs = [doc for doc in examples["article"]]
    
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

    

In [9]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[0, 574, 4524, 6, 1156, 36, 1251, 43, 480, 3268, 10997, 999, 3028, 7312, 20152, 3077, 899, 7, 10, 431, 984, 844, 153, 1358, 4006, 4, 134, 153, 43, 13016, 25, 37, 4072, 504, 15, 302, 6, 53, 37, 9838, 5, 418, 351, 75, 2471, 10, 8921, 15, 123, 4, 3028, 7312, 20152, 25, 3268, 10997, 11, 22, 29345, 10997, 8, 5, 9729, 9, 5, 5524, 113, 598, 5, 10208, 9, 20445, 6730, 1952, 198, 5, 232, 6, 5, 664, 2701, 161, 37, 34, 117, 708, 7, 856, 3961, 1334, 39, 1055, 409, 15, 1769, 1677, 6, 4076, 8, 6794, 1799, 4, 22, 100, 218, 75, 563, 7, 28, 65, 9, 167, 82, 54, 6, 25, 1010, 25, 51, 1004, 504, 6, 6017, 907, 1235, 10, 2232, 1612, 512, 2783, 50, 402, 1122, 60, 37, 174, 41, 2059, 33242, 656, 42, 353, 4, 22, 100, 218, 75, 206, 38, 581, 28, 1605, 31879, 4, 22, 133, 383, 38, 101, 2159, 32, 383, 14, 701, 59, 158, 2697, 480, 2799, 8, 32570, 8, 37206, 72, 497, 504, 6, 7312, 20152, 40, 28, 441, 7, 23104, 11, 10, 10297, 6, 907, 10, 4076, 11, 10, 8881, 50, 192, 5, 8444, 822, 22, 40534, 523, 35, 4657, 3

In [10]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)




In [11]:
sentence = "Does the tokenizer work I wonder?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0], 
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([    0, 27847,     5, 19233,  6315,   173,    38,  5170,   116,     2])

DECODED SENTENCE:
Does the tokenizer work I wonder?


It does.

#### Test out pre-trained BART

In [13]:
example_indices = [20, 50]
dash_line = 100 * '-'


for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['article']
    summary = dataset['test'][index]['highlights']

    prefix = 'Provide a very short summary of the following article: '
    
    inputs = tokenizer(dialogue, max_length=512, truncation=True, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=250,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'Pre-Trained BART GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

----------------------------------------------------------------------------------------------------
Example  1
----------------------------------------------------------------------------------------------------
INPUT PROMPT:
Norfolk, Virginia (CNN)The second mate of the Houston Express probably couldn't believe what he was seeing. Hundreds of miles from land there was a small boat nearby. At first it looked abandoned. It was in bad shape, listing to one side. The crew of the 1,000-foot long container ship thought it was a yacht that had wrecked. Incredibly, as they got closer, they saw there was a man on it, signaling for help. "He was moving, walking around, waving to us and in surprisingly good condition," Capt. Thomas Grenz told CNN by phone Friday. That man, Louis Jordan, 37, had an amazing story. He'd been drifting on the 35-foot Pearson sailboat for more than two months since leaving Conway, South Carolina, to fish in the ocean. Just a few days into his trip, a storm capsized h

In [157]:
example_indices = [20, 50]

for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['article']
    summary = dataset['test'][index]['highlights']

    prefix = 'Summarize the article in one or two sentences? :  '
    
    inputs = tokenizer(prefix + dialogue, max_length=512, truncation=True, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=250,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'Pre-Trained BART GENERATION - WITH PROMPT ENGINEERING:\n{output}\n')

----------------------------------------------------------------------------------------------------
Example  1
----------------------------------------------------------------------------------------------------
INPUT PROMPT:
Norfolk, Virginia (CNN)The second mate of the Houston Express probably couldn't believe what he was seeing. Hundreds of miles from land there was a small boat nearby. At first it looked abandoned. It was in bad shape, listing to one side. The crew of the 1,000-foot long container ship thought it was a yacht that had wrecked. Incredibly, as they got closer, they saw there was a man on it, signaling for help. "He was moving, walking around, waving to us and in surprisingly good condition," Capt. Thomas Grenz told CNN by phone Friday. That man, Louis Jordan, 37, had an amazing story. He'd been drifting on the 35-foot Pearson sailboat for more than two months since leaving Conway, South Carolina, to fish in the ocean. Just a few days into his trip, a storm capsized h

The original model struggles to reduce the summarisations to below 3 or 4 sentences. The CNN_daily mail dataset contains highlights that summarise the article in 1 or two sentences. 

With some basic prompt engineering the model still stuggles to reduce this further. This could possibly be improved with some fine tuning which is what we will try now.

### First we should get some baseline metrics for the original model

In [32]:
# Set up evaluation metric 
import evaluate
rouge = evaluate.load("rouge")


In [54]:
# Selecting a su

articles = dataset['test'][0:100]['article']
highlights = dataset['test'][0:100]['highlights']

original_model_summaries = []
instruct_model_summaries = []

for _, article in enumerate(articles):
    prompt = f"""
    'Summarize in one or two sentences the following article: '

    {article}

    Summary: """
    
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).input_ids

    # Generate summary
    original_model_outputs = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    
zipped_summaries = list(zip(highlights, original_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['Highlights', 'original_model_summaries'])
df


100
100


,Highlights,original_model_summaries
0,Membership gives the ICC jurisdiction over all...,Palestiniania. The move was made official. The...
1,"Theia, a bully breed mix, was apparently hit b...",A. The dog was hit by a car. The dog was burie...
2,Mohammad Javad Zarif has spent more time with ...,Moh. He is Iran's foreign minister. He is know...
3,17 Americans were exposed to the Ebola virus w...,The. The five were exposed to Ebola in Sierra ...
4,Student is no longer on Duke University campus...,The. The student admitted to hanging the noose...
...,...,...
95,"Manuscript of ""American Pie"" lyrics is sold to...",Don. Don McLean's song is 44-year. Christie's ...
96,Letourneau Fualaau had a sexual relationship w...,Mary. Mary Kay Kay Kay Letourneau Letour-Letou...
97,"Don McLean's ""American Pie"" lyrics auctioned f...",The. The song is one of the most dissected in ...
98,Mindy Kaling's brother Vijay Chokalingam prete...,Vay Chokalingam applied to St. Chokalingam pos...


In [59]:
print(df['original_model_summaries'])

0     Palestiniania. The move was made official. The...
1     A. The dog was hit by a car. The dog was burie...
2     Moh. He is Iran's foreign minister. He is know...
3     The. The five were exposed to Ebola in Sierra ...
4     The. The student admitted to hanging the noose...
                            ...                        
95    Don. Don McLean's song is 44-year. Christie's ...
96    Mary. Mary Kay Kay Kay Letourneau Letour-Letou...
97    The. The song is one of the most dissected in ...
98    Vay Chokalingam applied to St. Chokalingam pos...
99    T,. Travolt: "I've been so happy with my (Scie...
Name: original_model_summaries, Length: 100, dtype: object


In [55]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=highlights[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)



print('ORIGINAL MODEL:')
print(original_model_results)


ORIGINAL MODEL:
{'rouge1': np.float64(0.19090482977059542), 'rouge2': np.float64(0.0748375366841095), 'rougeL': np.float64(0.15159647802571558), 'rougeLsum': np.float64(0.15146151773990352)}


In [24]:
dataset['test']

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 11490
})

In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

In [119]:
tokenized_dataset_reduced = tokenized_datasets.filter(lambda example, index: index % 25 == 0, with_indices=True)

Filter: 100%|██████████| 11490/11490 [00:05<00:00, 1954.81 examples/s]


In [120]:
tokenized_dataset_reduced

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11485
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 535
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 460
    })
})

In [121]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 112361472
all model parameters: 406290432
percentage of trainable model parameters: 27.66%


In [122]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [123]:
# Load model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

C:\Users\tom_r\anaconda3\envs\Generative_AI\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [124]:
# Freeze all encoder layers except the last 2
for i, layer in enumerate(model.model.encoder.layers):
    if i < len(model.model.encoder.layers) - 2:
        for param in layer.parameters():
            param.requires_grad = False

# Freeze all decoder layers except the last 2
for i, layer in enumerate(model.model.decoder.layers):
    if i < len(model.model.decoder.layers) - 2:
        for param in layer.parameters():
            param.requires_grad = False


In [125]:
print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 112361472
all model parameters: 406290432
percentage of trainable model parameters: 27.66%


In [126]:
output_dir = f'./article-highlight-training-{str(int(time.time()))}'


training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,  # Lower learning rate
    per_device_train_batch_size=4,  # Reduce batch size to fit memory
    gradient_accumulation_steps=2,  # Adjust gradient accumulation
    num_train_epochs=3,  # Increase epochs to allow for better learning
    warmup_steps=50,  # Reduce warmup steps
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_reduced['train'],
    eval_dataset=tokenized_dataset_reduced['validation'],
    data_collator=data_collator 
)


C:\Users\tom_r\anaconda3\envs\Generative_AI\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [127]:
trainer.train()

Step,Training Loss,Validation Loss
500,1.331900,1.575234
1000,1.464800,1.563635
1500,1.325300,1.573067
2000,1.346500,1.579925
2500,1.326000,1.585557
3000,1.198100,1.588271
3500,1.401700,1.588427
4000,1.279500,1.592092


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

TrainOutput(global_step=4308, training_loss=1.3684610853398853, metrics={'train_runtime': 7019.9207, 'train_samples_per_second': 4.908, 'train_steps_per_second': 0.614, 'total_flos': 3.732646362434765e+16, 'train_loss': 1.3684610853398853, 'epoch': 3.0})

## Test out finetuned model

In [131]:
# Load finetuned model
finetuned_model = BartForConditionalGeneration.from_pretrained(r"C:\Users\tom_r\Desktop\Generative-AI\LLM_Projects\article-highlight-training-1726671662\checkpoint-4000")


In [133]:
example_indices = [20, 50, 75, 100, 200, 250, 300, 400]

for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['article']
    summary = dataset['test'][index]['highlights']

    prefix = 'Summarize in one or two sentences the following article: '
    
    inputs = tokenizer(prefix + dialogue, max_length=512, truncation=True, return_tensors='pt')
    output = tokenizer.decode(
        finetuned_model.generate(
            inputs["input_ids"], 
            max_new_tokens=250,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'Finetuned BART model:\n{output}\n')

----------------------------------------------------------------------------------------------------
Example  1
----------------------------------------------------------------------------------------------------
INPUT PROMPT:
Norfolk, Virginia (CNN)The second mate of the Houston Express probably couldn't believe what he was seeing. Hundreds of miles from land there was a small boat nearby. At first it looked abandoned. It was in bad shape, listing to one side. The crew of the 1,000-foot long container ship thought it was a yacht that had wrecked. Incredibly, as they got closer, they saw there was a man on it, signaling for help. "He was moving, walking around, waving to us and in surprisingly good condition," Capt. Thomas Grenz told CNN by phone Friday. That man, Louis Jordan, 37, had an amazing story. He'd been drifting on the 35-foot Pearson sailboat for more than two months since leaving Conway, South Carolina, to fish in the ocean. Just a few days into his trip, a storm capsized h

### Multi-shot inference

Finetuning the model has shown little improvement in shortening the length of the summary to a highlight. I think showing the model some examples during the prompt might aid the model.


In [144]:
# Load model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

C:\Users\tom_r\anaconda3\envs\Generative_AI\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [145]:
tokenized_dataset_reduced

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11485
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 535
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 460
    })
})

In [146]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        article = dataset['test'][index]['article']
        highlight = dataset['test'][index]['highlights']
        
        prompt += f"""
Article:

{article}

Summarize the article in one or two sentences? 
{highlight}


"""
    
    article = dataset['test'][example_index_to_summarize]['article']
    
    prompt += f"""
Article:

{article}

Summarize the article in one or two sentences?
"""
        
    return prompt

In [147]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Article:

(CNN)A high temperature of 63.5 degrees Fahrenheit might sound like a pleasant day in early spring -- unless you're in Antarctica. The chilly continent recorded the temperature (15.5 degrees Celsius) on March 24, possibly the highest ever recorded on Antarctica, according to the Weather Underground. The temperature was recorded at Argentina's Esperanza Base on the northern tip of the Antarctica Peninsula, according to CNN affiliate WTNH. (Note to map lovers: The Argentine base is not geographically part of the South American continent.) The World Meteorological Organization, a specialized United Nations agency, is in the process of setting up an international ad-hoc committee of about 10 blue-ribbon climatologists and meteorologists to begin collecting relevant evidence, said Randy Cerveny, the agency's lead rapporteur of weather and climate extremes and Arizona State University professor of geographical sciences. The committee will examine the equipment used to measure the 

In [152]:
highlight = dataset['test'][example_index_to_summarize]['highlights']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{highlight}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Aaron Hernandez has been found guilty in Odin Lloyd's death, but his troubles are not over . He also faces murder charges in Suffolk County, Massachusetts, but trial was postponed . In addition, Hernandez will face two civil lawsuits; one is in relation to Suffolk County case .

----------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
The temperature was recorded at Argentina's Esperanza Base on the northern tip of the Antarctica Peninsula. The World Meteorological Organization is in the process of setting up an international ad-hoc committee of about 10 blue-ribbon climatologists and meteorologists. The committee will examine the equipment used to measure the temperature, whether it was in good working order.


In [153]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Article:

(CNN)A high temperature of 63.5 degrees Fahrenheit might sound like a pleasant day in early spring -- unless you're in Antarctica. The chilly continent recorded the temperature (15.5 degrees Celsius) on March 24, possibly the highest ever recorded on Antarctica, according to the Weather Underground. The temperature was recorded at Argentina's Esperanza Base on the northern tip of the Antarctica Peninsula, according to CNN affiliate WTNH. (Note to map lovers: The Argentine base is not geographically part of the South American continent.) The World Meteorological Organization, a specialized United Nations agency, is in the process of setting up an international ad-hoc committee of about 10 blue-ribbon climatologists and meteorologists to begin collecting relevant evidence, said Randy Cerveny, the agency's lead rapporteur of weather and climate extremes and Arizona State University professor of geographical sciences. The committee will examine the equipment used to measure the 

In [155]:
summary = dataset['test'][example_index_to_summarize]['highlights']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

IndexError: index out of range in self